**Dataset Link: [UBC Ovarian Cancer Subtype Classification and Outlier Detection](https://www.kaggle.com/competitions/UBC-OCEAN/data)**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
# Update the DataFrame to include the correct filenames
df['image_id'] = df['image_id'].astype(str) + '_thumbnail.png'

# Show the first few rows of the updated DataFrame
df.head()


,image_id,label,image_width,image_height,is_tma
0,4_thumbnail.png,HGSC,23785,20008,False
1,66_thumbnail.png,LGSC,48871,48195,False
2,91_thumbnail.png,HGSC,3388,3388,True
3,281_thumbnail.png,LGSC,42309,15545,False
4,286_thumbnail.png,EC,37204,30020,False


In [ ]:
class_labels=['CC', 'EC', 'HGSC', 'LGSC', 'MC']
true_label = 2
predicted_class =

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
df['label'] = df['label'].astype('category')

# If you also want to access the numerical representation of the categories, you can use cat.codes
df['label_encoded'] = df['label'].cat.codes

# Display the updated DataFrame
df

,image_id,label,image_width,image_height,is_tma,label_encoded
0,4_thumbnail.png,HGSC,23785,20008,False,2
1,66_thumbnail.png,LGSC,48871,48195,False,3
2,91_thumbnail.png,HGSC,3388,3388,True,2
3,281_thumbnail.png,LGSC,42309,15545,False,3
4,286_thumbnail.png,EC,37204,30020,False,1
...,...,...,...,...,...,...
533,65022_thumbnail.png,LGSC,53355,46675,False,3
534,65094_thumbnail.png,MC,55042,45080,False,4
535,65300_thumbnail.png,HGSC,75860,27503,False,2
536,65371_thumbnail.png,HGSC,42551,41800,False,2


In [ ]:
image_path = '/content/drive/MyDrive/ubc'

In [ ]:
# Create train generator
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    zoom_range=0.1,
    validation_split=0.2,
    horizontal_flip=True)



In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory= image_path,  # Replace with your image folder path
    x_col='image_id',
    y_col='label',
    subset='training',
    batch_size=32,
    shuffle=True,
    class_mode='categorical',
    target_size=(243, 243))

Found 411 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 25 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
# Create validation generator
validation_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_path,
    x_col='image_id',
    y_col='label',
    target_size=(243, 243),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use 'validation' instead of 'training' for the validation set
)


Found 102 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 25 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
# Model Definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(243, 243, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(df['label'].nunique(), activation='softmax')  # Number of classes
])

In [ ]:
# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# Train the Model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

Epoch 1/10
13/13 [==============================] - 458s 35s/step - loss: 594.4471 - accuracy: 0.2725 - val_loss: 5.2040 - val_accuracy: 0.3137
Epoch 2/10
13/13 [==============================] - 185s 14s/step - loss: 2.4805 - accuracy: 0.2920 - val_loss: 1.5741 - val_accuracy: 0.3137
Epoch 3/10
13/13 [==============================] - 190s 15s/step - loss: 1.5272 - accuracy: 0.3528 - val_loss: 1.5327 - val_accuracy: 0.3922
Epoch 4/10
13/13 [==============================] - 177s 13s/step - loss: 1.5233 - accuracy: 0.3577 - val_loss: 1.5102 - val_accuracy: 0.3922
Epoch 5/10
13/13 [==============================] - 187s 14s/step - loss: 1.4847 - accuracy: 0.3869 - val_loss: 1.4778 - val_accuracy: 0.4412
Epoch 6/10
13/13 [==============================] - 183s 14s/step - loss: 1.4673 - accuracy: 0.4015 - val_loss: 1.4806 - val_accuracy: 0.4510
Epoch 7/10
13/13 [==============================] - 193s 15s/step - loss: 1.4557 - accuracy: 0.4112 - val_loss: 1.4738 - val_accuracy: 0.4412
Epoc

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input

# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer for classification (change num_classes to match your dataset)
num_classes = 5
predictions = Dense(num_classes, activation='softmax')(x)

# This is the model we will train
model2 = Model(inputs=base_model.input, outputs=predictions)

# Compile the model (should be done *after* setting layers to non-trainable)
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
# Train the Model
history = model2.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

Epoch 1/10
13/13 [==============================] - 523s 36s/step - loss: 2.0899 - accuracy: 0.3333 - val_loss: 21.6375 - val_accuracy: 0.4608
Epoch 2/10
13/13 [==============================] - 475s 36s/step - loss: 1.3199 - accuracy: 0.4550 - val_loss: 19.0228 - val_accuracy: 0.4608
Epoch 3/10
13/13 [==============================] - 465s 35s/step - loss: 1.2483 - accuracy: 0.4331 - val_loss: 25.3196 - val_accuracy: 0.2157
Epoch 4/10
13/13 [==============================] - 464s 35s/step - loss: 1.1202 - accuracy: 0.5255 - val_loss: 58.7756 - val_accuracy: 0.1863
Epoch 5/10
13/13 [==============================] - 457s 34s/step - loss: 1.1427 - accuracy: 0.5401 - val_loss: 10.7488 - val_accuracy: 0.1961
Epoch 6/10
13/13 [==============================] - 452s 34s/step - loss: 1.0065 - accuracy: 0.5742 - val_loss: 7.2575 - val_accuracy: 0.1667
Epoch 7/10
13/13 [==============================] - 521s 39s/step - loss: 0.9368 - accuracy: 0.6229 - val_loss: 5.9475 - val_accuracy: 0.0882
E

In [ ]:
#Testing the data using both the models

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
from PIL import Image
import numpy as np

# Increase the maximum number of pixels
Image.MAX_IMAGE_PIXELS = None

# Load and preprocess the test image
test_image_path = '/content/41.png'
img = image.load_img(test_image_path, target_size=(243, 243))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Normalize the image if it was done during training
img_array /= 255.0  # Only if your training images were scaled in this way

# Make a prediction
predictions = model.predict(img_array)

# Assuming a multi-class classification, get the class with the highest probability
predicted_class = np.argmax(predictions, axis=1)

# Print the predicted class
print("Predicted class:", predicted_class)


1/1 [==============================] - 0s 495ms/step
Predicted class: [2]


In [ ]:
# Load the trained model


# Load and preprocess the test image
test_image_path = '/content/41.png'
img = image.load_img(test_image_path, target_size=(243, 243))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Normalize the image if it was done during training
img_array /= 255.0

# Make a prediction
predictions = model2.predict(img_array)

# Assuming a multi-class classification, get the class with the highest probability
predicted_class = np.argmax(predictions, axis=1)

# Print the predicted class
print("Predicted class:", predicted_class)


1/1 [==============================] - 4s 4s/step
Predicted class: [2]


In [ ]:
from sklearn.metrics import f1_score, classification_report

# Define the true labels for the test data
y_true = [0, 1, 2, 3, 4]  # Example true labels for the test data

# Define the single predicted class for the test data
predicted_class = 2  # Example single predicted class

# Calculate the F1 score for the single predicted class against each true class
f1_per_class = f1_score(y_true, [predicted_class] * len(y_true), average=None)

# Print the F1 score for the single predicted class against each true class
for class_label, f1 in enumerate(f1_per_class):
    print(f"F1-score for predicted class {predicted_class} against true class {class_label}: {f1}")

F1-score for predicted class 2 against true class 0: 0.0
F1-score for predicted class 2 against true class 1: 0.0
F1-score for predicted class 2 against true class 2: 0.33333333333333337
F1-score for predicted class 2 against true class 3: 0.0
F1-score for predicted class 2 against true class 4: 0.0


In [ ]:
from sklearn.metrics import classification_report
class_labels=['CC', 'EC', 'HGSC', 'LGSC', 'MC']
true_label = 2
predicted_class = 2

# Convert class indices to class labels
predicted_label_name = class_labels[predicted_class]
true_label_name = class_labels[true_label] if isinstance(true_label, int) else true_label

# Generate a classification report
true_labels = [true_label_name]
predicted_labels = [predicted_label_name]

# Include all class labels in the report
all_class_indices = list(range(len(class_labels)))

# Print the classification report
print(classification_report(true_labels, predicted_labels, labels=all_class_indices, target_names=class_labels, zero_division=0))

              precision    recall  f1-score   support

          CC       0.00      0.00      0.00         0
          EC       0.00      0.00      0.00         0
        HGSC       0.00      0.00      0.00         0
        LGSC       0.00      0.00      0.00         0
          MC       0.00      0.00      0.00         0

   micro avg       0.00      0.00      0.00         0
   macro avg       0.00      0.00      0.00         0
weighted avg       0.00      0.00      0.00         0



/usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
/usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
/usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
/usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


In [ ]:
#The predicted label is 2 i.e HSGC thats the true value to be predicted by the model